In [22]:
import QuantLib as ql

In [23]:
#chris, haonan project#
class Price:
    calculation_date = ql.Date(31, 12, 2018)
    risk_free_rate = 0
    libor_rate = 0

    ##TODO
    #forward, maturity need to be passed in as parameters

    def __init__(self, risk_free_rate, libor_rate):
        self.risk_free_rate = risk_free_rate
        self.libor_rate = libor_rate

    def discountCurve(self):
        day_count = ql.Actual365Fixed()
        discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(self.calculation_date, self.risk_free_rate, day_count))
        return discount_curve

    def liborIndex(self):
        day_count = ql.Actual365Fixed()
        libor_curve = ql.YieldTermStructureHandle(ql.FlatForward(self.calculation_date, self.libor_rate, day_count))
        libor_index = ql.USDLibor(ql.Period(3, ql.Months), libor_curve) #?#
        return libor_index

    def fixFloatLegSchedule(self):
        calendar = ql.UnitedStates()
        settle_date = calendar.advance(self.calculation_date, 5, ql.Days) #?#
        maturity_date = calendar.advance(settle_date, 10, ql.Years) #?#
        fixed_leg_tenor = ql.Period(6, ql.Months) #?#
        fixed_schedule = ql.Schedule(settle_date, maturity_date, 
                                    fixed_leg_tenor, calendar,
                                    ql.ModifiedFollowing, ql.ModifiedFollowing,
                                    ql.DateGeneration.Forward, False)
        float_leg_tenor = ql.Period(3, ql.Months) #?#
        float_schedule = ql.Schedule (settle_date, maturity_date, 
                                    float_leg_tenor, calendar,
                                    ql.ModifiedFollowing, ql.ModifiedFollowing,
                                    ql.DateGeneration.Forward, False)
        return fixed_schedule, float_schedule

    def price(self, notional, fixed_rate, float_spread):
        return 0

In [24]:
class vanillaPrice(Price):

    def __init__(self, risk_free_rate, libor_rate):
        super().__init__(risk_free_rate, libor_rate)

    def price(self, notional, fixed_rate, float_spread):
        fixed_leg_daycount = ql.Actual365Fixed()
        float_leg_daycount = ql.Actual365Fixed()
        discount_curve = self.discountCurve()
        libor_index = self.liborIndex()
        fixed_schedule, float_schedule = self.fixFloatLegSchedule()
        ir_swap = ql.VanillaSwap(ql.VanillaSwap.Payer, notional, fixed_schedule, 
                    fixed_rate, fixed_leg_daycount, float_schedule,
                    libor_index, float_spread, float_leg_daycount )
        swap_engine = ql.DiscountingSwapEngine(discount_curve)
        ir_swap.setPricingEngine(swap_engine)
        return ir_swap

In [25]:
vanilla = vanillaPrice(0.01, 0.02)
price = vanilla.price(10000000, 0.025, 0.004)

In [26]:
print("Fixed Leg Cashflows:")
for i, cf in enumerate(price.leg(0)):
    print("%2d    %-18s  %10.2f"%(i+1, cf.date(), cf.amount()))

Fixed Leg Cashflows:
 1    July 8th, 2019       123972.60
 2    January 8th, 2020    126027.40
 3    July 8th, 2020       124657.53
 4    January 8th, 2021    126027.40
 5    July 8th, 2021       123972.60
 6    January 10th, 2022   127397.26
 7    July 8th, 2022       122602.74
 8    January 9th, 2023    126712.33
 9    July 10th, 2023      124657.53
10    January 8th, 2024    124657.53
11    July 8th, 2024       124657.53
12    January 8th, 2025    126027.40
13    July 8th, 2025       123972.60
14    January 8th, 2026    126027.40
15    July 8th, 2026       123972.60
16    January 8th, 2027    126027.40
17    July 8th, 2027       123972.60
18    January 10th, 2028   127397.26
19    July 10th, 2028      124657.53
20    January 8th, 2029    124657.53


In [27]:
for i, cf in enumerate(price.leg(1)):
    print(i+1, cf.date(), cf.amount())

RuntimeError: Missing USDLibor3M Actual/360 fixing for January 4th, 2019